<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/exercicios/Aula_3/Aula_3_Rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 1 - Fine-tunning do modelo

## Preparo do conjunto de dados

### Download do arquivo

In [1]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv 

--2023-03-15 14:01:58--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  --.-KB/s    in 0.1s    

2023-03-15 14:01:58 (79.7 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]



### Parsing do arquivo e criação das listas básicas

In [2]:
import csv

# Lists to store the queries, positive documents, and negative documents
queries = []
relevant_docs = []
non_relevant_docs = []

with open('msmarco_triples.train.tiny.tsv', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter="\t")
    for row in reader:
        queries.append(row[0])
        relevant_docs.append(row[1])
        non_relevant_docs.append(row[2])

# Print the first 5 queries, positive documents, and negative documents
print("Queries:", queries[:5])
print("Relevant documents:", relevant_docs[:5])
print("Non relevant documents:", non_relevant_docs[:5])
print(len(queries), len(relevant_docs), len(non_relevant_docs))

Queries: ['is a little caffeine ok during pregnancy', 'what fruit is native to australia', 'how large is the canadian military', 'types of fruit trees', 'how many calories a day are lost breastfeeding']
Relevant documents: ['We donâ\x80\x99t know a lot about the effects of caffeine during pregnancy on you and your baby. So itâ\x80\x99s best to limit the amount you get each day. If youâ\x80\x99re pregnant, limit caffeine to 200 milligrams each day. This is about the amount in 1Â½ 8-ounce cups of coffee or one 12-ounce cup of coffee.', 'Passiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sources list the fruit as edible, sweet and tasty, while others list the fruits as being bitter and inedible.assiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sources list the fruit as edible, sweet and tasty, while others l

### Split dos conjuntos de treinamento, validação e teste

In [10]:
import random 
import numpy as np

import torch
import numpy as np

seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


size = len(queries) * 2
all_indexes = list(range(size))
random.shuffle(all_indexes)

training_size = size * 80 // 100
test_size = size * 10 // 100
training_indexes = all_indexes[:training_size]
validation_indexes = all_indexes[training_size:-test_size]
test_indexes = all_indexes[-test_size:]

all_queries = np.array(queries * 2)
all_docs = np.array(relevant_docs + non_relevant_docs)
all_labels = np.array([True] * len(relevant_docs) + [False] * len(non_relevant_docs))

training_queries = all_queries[training_indexes].tolist()
training_docs = all_docs[training_indexes].tolist()
training_labels = all_labels[training_indexes].tolist()

validation_queries = all_queries[validation_indexes].tolist()
validation_docs = all_docs[validation_indexes].tolist()
validation_labels = all_labels[validation_indexes].tolist()

test_queries = all_queries[test_indexes].tolist()
test_docs = all_docs[test_indexes].tolist()
test_labels = all_labels[test_indexes].tolist()

# Print the first 5 queries, documents, and classifications
def print_triples(queries, docs, labels, quantity, prefix):
    for i in range(quantity):
        print(prefix + 'Query:', queries[i])
        print(prefix + 'Doc:', docs[i])
        print(prefix + 'Label:', labels[i])
        print()

quantity = 5
print(f'Training examples ({quantity} / {len(training_queries)}):')
print_triples(training_queries, training_docs, training_labels, 5, '\t')

print(f'Validation examples ({quantity} / {len(validation_queries)}):')
print_triples(validation_queries, validation_docs, validation_labels, 5, '\t')

print(f'Test examples ({quantity} / {len(test_queries)}):')
print_triples(test_queries, test_docs, test_labels, 5, '\t')

Training examples (5 / 17600):
	Query: what does a hyphen look like
	Doc: 1 Using the Hyphen Correctly A hyphen is a simple looking punctuation mark, just a small horizontal line that usually connects entire words together. 2  However, using a hyphen, or hyphenating, entails usâ¦. 3  When to Hyphenate Compound Nouns and Compound Words Many writers struggle whether they need a hyphen or not.
	Label: False

	Query: how many mg in a prescription dose of acetaminophen
	Doc: 1 Major Depressive Disorder: Starting daily dose is 40 mg and should be taken in two separate 20 mg doses. 2  In âmaintenance modeâ the recommended dose is 60 mg and can be taken all at once or as two separate 30 mg doses. 3  The maximum dose, while not recommended is 120 mg/day.he recommended Cymbalta dosage, based on the published Product Guide, are as follows: 1  Major Depressive Disorder: Starting daily dose is 40 mg and should be taken in two separate 20 mg doses.
	Label: False

	Query: gleam meaning
	Doc: Def

## Treinamento do modelo

### Instalação de biblioteca

In [12]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.9 MB/s eta 0:00:00


### Definição do tokenizador

In [13]:
from transformers import AutoTokenizer

model_name = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Definição do tamanho da sequência de tokens

In [25]:
max_length = max([len(tokenizer(f'{all_queries[idx]} [SEP] {all_docs[idx]}', truncation=True)['input_ids']) for idx in range(len(all_docs))])
max_length

290

### Criação dos datasets de treinamento, validação e teste

In [27]:
from torch.utils.data import Dataset, DataLoader

class ModelDataset(Dataset):
  def __init__(self, tokenizer, queries, docs, labels, max_length = 256):
    self.max_length = max_length
    self.tokenizer = tokenizer
    self.queries = queries
    self.docs = docs
    self.labels = labels

  def __len__(self):
    return len(self.queries)
    
  def __getitem__(self, idx):
    batch_encoding = self.tokenizer(f'{self.queries[idx]} [SEP] {self.docs[idx]}', truncation=True, max_length = self.max_length)
    return {
        'input_ids': batch_encoding['input_ids'],
        'attention_mask': batch_encoding['attention_mask'],
        'labels': int(self.labels[idx])
    }

In [33]:
max_seq_length = 256

dataset_training = ModelDataset(tokenizer, training_queries, training_docs, training_labels, max_length)
dataset_validation = ModelDataset(tokenizer, validation_queries, validation_docs, validation_labels, max_length)
dataset_test = ModelDataset(tokenizer, test_queries, test_docs, test_labels, max_length)

len(dataset_training), len(dataset_validation), len(dataset_test)

(17600, 2200, 2200)

In [39]:
print(dataset_training[100]['input_ids'])
print(tokenizer.decode(dataset_training[100]['input_ids']))
print(dataset_training[100]['labels'] == 1)

[101, 2054, 2515, 6046, 2184, 8093, 3446, 2812, 102, 8224, 2377, 2578, 5983, 6046, 1012, 1045, 2031, 3728, 4384, 2008, 2119, 2026, 26041, 1020, 1998, 2026, 2564, 1005, 1055, 26041, 1019, 2031, 2042, 5983, 6046, 2012, 2019, 8598, 2075, 3446, 1012, 1999, 2119, 11640, 1010, 8224, 2377, 2578, 2003, 2012, 1996, 2327, 1997, 1996, 6046, 8381, 3673, 2007, 2053, 6835, 3114, 2339, 1012, 102]
[CLS] what does battery 10hr rate mean [SEP] google play services eating battery. i have recently noticed that both my nexus 6 and my wife's nexus 5 have been eating battery at an alarming rate. in both phones, google play services is at the top of the battery consumption chart with no apparent reason why. [SEP]
False


### Criação dos dataloaders de treinamento, validação e teste

In [44]:
batch_size = 32

from transformers import BatchEncoding

collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
dataloader_training = DataLoader(dataset_training, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_validation = DataLoader(dataset_validation, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_test = DataLoader(dataset_validation, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

### Função de avaliação do modelo

In [45]:
# We first define the evaluation function to measure accuracy and loss
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

### Treinamento e validação do modelo

In [47]:
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from statistics import mean

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 5
num_training_steps = epochs * len(dataloader_training)

# First validation to check if evaluation code is working and accuracy is random as expected 
evaluate(model=model, dataloader=dataloader_validation, set_name='Valid')

# Warm up is important to stabilize training.
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Training loop
for epoch in tqdm(range(epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_training, mininterval=0.5, desc='Train', disable=False):
        optimizer.zero_grad()
        outputs = model(**batch.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.cpu().item())

    print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
    evaluate(model=model, dataloader=dataloader_validation, set_name='Valid')

device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Valid:   0%|          | 0/69 [00:00<?, ?it/s]

Valid loss: 0.693; Valid accuracy: 0.501


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Valid loss: 0.273; Valid accuracy: 0.919


Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.04


Valid:   0%|          | 0/69 [00:00<?, ?it/s]

Valid loss: 0.303; Valid accuracy: 0.921


### Teste do modelo

In [48]:
evaluate(model=model, dataloader=dataloader_test, set_name='Test')

Test:   0%|          | 0/69 [00:00<?, ?it/s]

Test loss: 0.303; Test accuracy: 0.921


### Salvamento do modelo no Google Drive

In [49]:
from google.colab import drive
drive.mount('/content/drive')
LOCAL_MODELO = '/content/drive/My Drive/IA368-DD/Aula3/'

model.save_pretrained(LOCAL_MODELO)
tokenizer.save_pretrained(LOCAL_MODELO)

Mounted at /content/drive


('/content/drive/My Drive/IA368-DD/Aula2/tokenizer_config.json',
 '/content/drive/My Drive/IA368-DD/Aula2/special_tokens_map.json',
 '/content/drive/My Drive/IA368-DD/Aula2/vocab.txt',
 '/content/drive/My Drive/IA368-DD/Aula2/added_tokens.json',
 '/content/drive/My Drive/IA368-DD/Aula2/tokenizer.json')

# Parte 2 - Rankeamento básico BM25

## Carga das bibliotecas

In [1]:
# atualização da versão do scikit-learn por dependência do pyserini

!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 116.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [3]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 16.0 MB/s eta 0:00:00


## Carga da engine de busca com o dataset MsMarco Passage indexado

In [4]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

lucene-index.msmarco-v1-passage.20221004.252b5e.tar.gz: 2.02GB [01:13, 29.6MB/s]                            


## Carga dos julgamentos do TREC 2020

In [5]:
from pyserini.search import get_qrels

# Carrega a lista de queries do TREC DL 2020
qrels_ids = list(get_qrels('dl20-passage').keys())

# Teste
qrels_idx = 10
query_id = qrels_ids[qrels_idx]
print('Query id:', query_id, '\nJulgamentos:')
print(get_qrels('dl20-passage')[query_id])

Query id: 174463 
Julgamentos:
{1013072: '2', 1013078: '2', 1024759: '1', 1024760: '1', 1024761: '2', 1024762: '2', 1024764: '2', 1024765: '2', 1024766: '0', 1024767: '2', 102528: '0', 1035028: '0', 1177464: '0', 1185676: '0', 1295676: '0', 1409812: '0', 1432589: '0', 1432591: '2', 1432592: '0', 1432593: '0', 1432594: '1', 1432595: '2', 1432596: '2', 1479952: '0', 1567953: '1', 1567955: '1', 1567956: '2', 1567957: '0', 1567958: '2', 1567959: '1', 1567960: '2', 1567961: '0', 1634041: '0', 1634777: '0', 175720: '0', 1762600: '0', 1788353: '2', 1788354: '2', 1788355: '2', 1788356: '0', 1788357: '2', 1788358: '2', 1788359: '2', 1788360: '2', 1788361: '2', 1788362: '2', 1835369: '2', 1835370: '2', 1835371: '2', 1835373: '0', 1835375: '2', 1835377: '2', 1854163: '1', 1854164: '1', 1854165: '2', 1854166: '2', 1854167: '2', 1854169: '2', 1854170: '1', 1970971: '0', 1973697: '1', 1973702: '0', 2037110: '0', 2157104: '0', 2157109: '0', 2157112: '2', 2216449: '0', 2295540: '0', 2348940: '0', 2348

## Carga das queries

In [6]:
from pyserini.search import get_topics

# Carrega a lista de queries do TREC DL 2020
topics = get_topics('dl20')

# teste
topics[query_id]['title']

'dog day afternoon meaning'

In [7]:
import json

doc = searcher.doc('7157715')
json_doc = json.loads(doc.raw())

json_doc['id'], json_doc['contents']

('7157715',
 'A Lobster Roll is a bread roll filled with bite-sized chunks of lobster meat. Lobster Rolls are made on the Atlantic coast of North America, from the New England area of the United States on up into the Maritimes areas of Canada.')

## Execução da consulta sobre as queries 

In [9]:
K_base = 1000
results_idx = range(K_base)

results_bm25 = {}
for query_id in qrels_ids:
    hits = searcher.search(topics[query_id]['title'], k = K_base)
    results_bm25[query_id] = [int(hits[ord].docid) for ord in results_idx]

len(qrels_ids), len(results_bm25), len(list(results_bm25.items())[3][1])

(54, 54, 1000)

In [10]:
print('query_id [10 first hits (docs)]:')
for query_id, hits in results_bm25.items():
    print(query_id, hits[:10])

query_id [10 first hits (docs)]:
23849 [4348282, 2674124, 7119957, 8133127, 542113, 2516458, 4834498, 436721, 6667419, 8246990]
42255 [8746197, 8746189, 8746188, 3723425, 5550457, 182410, 8372967, 5997797, 6784918, 7123389]
47210 [2042918, 6180504, 2338385, 2545998, 2546001, 2546002, 6180507, 5207329, 7562282, 7367873]
67316 [6680964, 4163705, 8431637, 6764671, 1974375, 450305, 8431632, 7630885, 3810719, 450309]
118440 [7035722, 1569786, 6750548, 6750552, 6750547, 1445681, 1569787, 2975216, 2975214, 4403471]
121171 [8260192, 451171, 451173, 259437, 8643275, 259436, 2550811, 8056821, 7165126, 451172]
135802 [8692623, 8692626, 8692628, 8692619, 8692624, 8692625, 8692627, 8692621, 8692622, 7327478]
141630 [6290795, 1157249, 8319991, 5126207, 4098919, 4098924, 8319993, 8319990, 8319989, 3139270]
156498 [5606985, 4273419, 4273412, 8146073, 3301559, 8823842, 494576, 3397582, 4273410, 3288004]
169208 [4859433, 721607, 721610, 85987, 4859432, 8303019, 4762135, 4762140, 4202275, 4580137]
174463

## Cálculo do IDCG@10

In [11]:
import math

K_relevance = 10

def dcg(evals):
    dcg = 0
    for i, eval in enumerate(evals):
        dcg += (2**int(eval)-1) * math.log(2) / math.log(i+2)
    return dcg

idcg = {}
print(f'query_id [{K_relevance} best evaluations] idcg:')
for query_id, hits in results_bm25.items():
    ideal_results = sorted([int(relevance) for relevance in get_qrels('dl20-passage')[query_id].values()], reverse=True)[:K_relevance]
    idcg[query_id] = dcg(ideal_results)
    print(query_id, ideal_results, idcg[query_id])

query_id [10 best evaluations] idcg:
23849 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 31.804915366618417
42255 [3, 3, 3, 2, 2, 2, 1, 0, 0, 0] 18.7710512655814
47210 [3, 3, 3, 3, 3, 3, 3, 2, 2, 2] 28.182676571548026
67316 [3, 3, 3, 3, 3, 3, 2, 2, 2, 2] 26.849343238214693
118440 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 31.804915366618417
121171 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 13.630678014265037
135802 [2, 2, 2, 2, 2, 2, 2, 2, 2, 1] 13.052548361629261
141630 [3, 3, 2, 2, 2, 2, 2, 2, 2, 2] 20.154397028550864
156498 [3, 2, 2, 2, 2, 2, 2, 1, 1, 1] 15.819558616729841
169208 [3, 3, 3, 3, 2, 2, 2, 2, 2, 2] 23.877103260844436
174463 [3, 2, 2, 2, 2, 2, 2, 2, 2, 2] 17.630678014265037
258062 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 31.804915366618417
324585 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 31.804915366618417
330975 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 31.804915366618417
332593 [3, 3, 3, 3, 3, 3, 2, 2, 2, 2] 26.849343238214693
336901 [3, 3, 3, 0, 0, 0, 0, 0, 0, 0] 14.9165082750002
390360 [3, 3, 3, 3, 3, 3, 2, 2, 2, 2] 26.849343238214693
40

## Avaliação dos resultados

In [12]:
dcg_queries = {}
ndcg = {}

print(f'query_id [{K_relevance} bm25 evaluations] dcg --> ndcg:')
for query_id, hits in results_bm25.items():
    retrieved_results = [int(get_qrels('dl20-passage')[query_id].get(doc_id, 0)) for doc_id in hits[:10]]
    dcg_queries[query_id] = dcg(retrieved_results)
    ndcg[query_id] = dcg_queries[query_id]/idcg[query_id]
    print(query_id, retrieved_results, dcg_queries[query_id], '-->', ndcg[query_id])

query_id [10 bm25 evaluations] dcg --> ndcg:
23849 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 0.2890648263178878 --> 0.009088684028421673
42255 [3, 3, 3, 0, 0, 0, 0, 0, 0, 0] 14.9165082750002 --> 0.794654921770477
47210 [3, 1, 2, 3, 3, 3, 0, 1, 0, 1] 17.951615323586772 --> 0.6369734002380002
67316 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0] 7.0 --> 0.26071401217877443
118440 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0] 4.416508275000202 --> 0.1388624438735545
121171 [1, 1, 1, 0, 1, 1, 2, 1, 0, 2] 5.056649103653413 --> 0.37097561092422787
135802 [2, 2, 2, 2, 2, 2, 2, 2, 2, 0] 12.763483535311373 --> 0.9778537632415402
141630 [1, 2, 1, 1, 1, 1, 2, 1, 1, 1] 6.472085511897926 --> 0.3211252364796388
156498 [0, 2, 0, 0, 3, 0, 0, 0, 1, 0] 4.901788907020144 --> 0.3098562371921236
169208 [2, 0, 0, 2, 2, 0, 2, 0, 0, 0] 6.4525880959238044 --> 0.2702416631294328
174463 [2, 3, 0, 0, 0, 0, 0, 0, 0, 0] 7.416508275000202 --> 0.4206592774820958
258062 [3, 0, 0, 0, 0, 0, 0, 0, 0, 3] 9.023453784225214 --> 0.28371255449703187
324585 [3, 0, 0, 0

In [13]:
import numpy as np
from statistics import mean

mean(list(ndcg.values()))

0.433924103352141

# Parte 3 - Rerank

## Carga do modelo ajustado

In [20]:
from google.colab import drive
drive.mount('/content/drive')
LOCAL_MODELO = '/content/drive/My Drive/IA368-DD/Aula3/'

Mounted at /content/drive


In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODELO)
model = AutoModelForSequenceClassification.from_pretrained(LOCAL_MODELO).to(device)

device: cuda


## Criação do dataset do rerank

In [22]:
from torch.utils.data import Dataset, DataLoader

class RerankDataset(Dataset):
  def __init__(self, tokenizer, queries, hits, max_length = 300, K = 1000):
    self.max_length = max_length
    self.tokenizer = tokenizer
    self.queries = queries
    self.hits = hits
    self.K = K

  def __len__(self):
    return self.K * len(self.queries)
    
  def __getitem__(self, idx):
    query_id = self.queries[idx // self.K]
    doc_id = self.hits[query_id][idx % self.K]
    doc = searcher.doc(doc_id)
    json_doc = json.loads(doc.raw())
    sentence = f"{topics[query_id]['title']} [SEP] {json_doc['contents']}"

    batch_encoding = self.tokenizer(sentence, truncation=True, max_length = self.max_length)
    return {
        'input_ids': batch_encoding['input_ids'],
        'attention_mask': batch_encoding['attention_mask']
    }

max_length = 300 
rerank_dataset = RerankDataset(tokenizer, qrels_ids, results_bm25, max_length, K_base)
len(rerank_dataset)

54000

## Criação do dataloader do rerank

In [23]:
batch_size = 32

from transformers import BatchEncoding

collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
rerank_dataloader = DataLoader(rerank_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

## Execução do modelo (previsões)

In [34]:
from tqdm.auto import tqdm

score = {}

losses = []
correct = 0
model.eval()
idx = 0
with torch.no_grad():
    for batch in tqdm(rerank_dataloader, mininterval=0.5, desc='Rerank', disable=False):
        for logit in model(**batch.to(device)).logits:
            query_id = qrels_ids[idx // K_base]
            doc_id = results_bm25[query_id][idx % K_base]
            if query_id not in score:
                score[query_id] = {}
            score[query_id][doc_id] = logit[1].item()
            idx += 1

Rerank:   0%|          | 0/1688 [00:00<?, ?it/s]

## Seleção dos melhores resultados

In [47]:
results_rerank = {}

for query_id in qrels_ids:
    scores = [(score_doc, doc_id) for doc_id, score_doc in score[query_id].items()]
    best_docs = [doc_id for _, doc_id in sorted(scores, reverse=True)[:K_relevance]]
    print(query_id, best_docs)
    results_rerank[query_id] = best_docs

23849 [8060949, 859115, 7559527, 5476361, 6020424, 8060634, 8404581, 4935403, 7607463, 1741530]
42255 [6317267, 490581, 6030149, 2040836, 1956828, 6030146, 5571058, 1151785, 6560214, 94729]
47210 [2344224, 2566994, 5501443, 8740517, 7832622, 2443840, 5691600, 6202430, 1618520, 6424417]
67316 [3334787, 6449837, 7111673, 2603862, 2173090, 3692817, 7330185, 91147, 8050859, 6425678]
118440 [4052287, 8151355, 1711898, 1227893, 7615787, 1440029, 1918708, 6977842, 7317711, 3191067]
121171 [1604088, 2029254, 3216351, 451176, 984406, 5237908, 1218495, 5434440, 4916620, 337257]
135802 [7361462, 4682759, 4346786, 8412720, 3720219, 2198707, 8085509, 4056505, 3492361, 8117361]
141630 [4261424, 2673568, 1980895, 6256350, 3604692, 7019485, 4671690, 987958, 5913414, 4422343]
156498 [2206872, 8167576, 5670871, 8528404, 608667, 8426643, 2149492, 3698261, 7456147, 2149494]
169208 [314934, 3059075, 2256971, 4500987, 6903555, 1224354, 3120279, 6109859, 5175725, 8416223]
174463 [2505354, 102528, 8349544, 32

## Avaliação dos resultados

In [48]:
dcg_rerank = {}
ndcg_rerank = {}

print(f'query_id [{K_relevance} bm25 evaluations] dcg --> ndcg:')
for query_id, hits in results_rerank.items():
    retrieved_results = [int(get_qrels('dl20-passage')[query_id].get(doc_id, 0)) for doc_id in hits[:10]]
    dcg_rerank[query_id] = dcg(retrieved_results)
    ndcg_rerank[query_id] = dcg_rerank[query_id]/idcg[query_id]
    print(query_id, retrieved_results, dcg_rerank[query_id], '-->', ndcg_rerank[query_id])

query_id [10 bm25 evaluations] dcg --> ndcg:
23849 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
42255 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
47210 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0] 0.33333333333333337 --> 0.011827596732591816
67316 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
118440 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
121171 [0, 0, 0, 2, 0, 1, 0, 0, 0, 0] 1.6482368613282012 --> 0.12092112069577589
135802 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
141630 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 0.2890648263178878 --> 0.014342519198584631
156498 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
169208 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
174463 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
258062 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
324585 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
330975 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
332593 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
336901 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0 --> 0.0
390360 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] 0.5 --> 0.018622429441341

In [49]:
mean(list(ndcg_rerank.values()))

0.024253617635682723

# Comparação dos métodos de busca

In [52]:
print('nDCG@10 (BM25):  ', mean(list(ndcg.values())))
print('nDCG@10 (Rerank):', mean(list(ndcg_rerank.values())))

nDCG@10 (BM25):   0.433924103352141
nDCG@10 (Rerank): 0.024253617635682723
